In [148]:
from  ipywidgets  import interact
import numpy as np
import pandas as pd
from datetime import date
from datetime import datetime
import mibian
import glob, os

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import HoverTool
output_notebook()

Loading BokehJS ...

In [149]:
df = pd.read_csv("portfolio_records.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [150]:
eq_plot = figure(title="Portfolio Equity", plot_height=500, plot_width=900, y_range=(9800,18000), x_axis_type="datetime")
r = eq_plot.line(source=df, x='timestamp', y='portfolio_equity')
eq_plot.add_tools(HoverTool(tooltips=[
        ('date','@timestamp{%F}'),
        ('equity','@portfolio_equity')
    ],
    formatters={
        'timestamp':'datetime', 
        'equity':'printf',
        
    }
))

show(eq_plot, notebook_handle=True)

monthly_returns = []
returns_df = pd.DataFrame()
cur_month = 1
prev_equity = df.at[0,'portfolio_equity']

for index, row in df.iterrows():
    if index == (len(df.index) - 1) or df.at[index + 1,'timestamp'].month != cur_month:
        perc_change = (row['portfolio_equity'] - prev_equity) / prev_equity * 100    
        perc_change = float("{0:.2f}".format(perc_change))
        returns_df.at['2017', df.at[index,'timestamp'].strftime("%B")] = perc_change
        prev_equity = row['portfolio_equity']
        cur_month += 1
        
def color_negative_red(val):
    return 'background-color: #FBA38D' if val < 0 else 'background-color: #D0FB8D'  
returns_df = returns_df.style.applymap(color_negative_red)

returns_df

,January,February,March,April,May,June,July,August,September,October,November,December
2017,12.25,15.59,2.55,5.17,-1.47,3.72,0.36,6.07,2.28,3.28,1.66,0.33


In [237]:
class option(object):
   def  __init__(self, strike,price,type, units, action, exp_date):
        self.strike = strike
        self.type = type
        self.units = units
        self.action = action
        self.exp_date = exp_date
        self.price = price
        
   def update(self, price, dte, underlying):
        self.dte = dte
        self.price = price
        optDat = mibian.BS([underlying, self.strike, 1.45, dte], putPrice=(price/1000))
        self.impliedVolatility = optDat.impliedVolatility
   
underlying  = [p/10 for p in range(350, 700)]
def calc_vals(options):
    vals = []
    for val in underlying:
        tot_val = 0.0    
        for contract in options:            
            if val > contract.strike:
                tot_val += (0 -  contract.price) * contract.action * contract.units;
            else:
                tot_val += (((contract.strike - val)*1000) - contract.price) * contract.action * contract.units
        vals.append(tot_val)
    return vals
def calc_theoVals(options, trade_timestamp):
    theoVals = []
    for val in underlying:
        tot_val = 0.0    
        for contract in options:
            calc = mibian.BS([val, contract.strike, 1.45, contract.dte], volatility = contract.impliedVolatility)
            tot_val += ((calc.putPrice * 1000) - contract.price) * contract.action * contract.units
        theoVals.append(tot_val)   
    return theoVals   

In [225]:
df_exec_records = pd.read_csv("execution_records.csv")
df_exec_records["expiration_date"] = pd.to_datetime(df_exec_records["expiration_date"])
df_exec_records["timestamp"] = pd.to_datetime(df_exec_records["timestamp"])

contract_data = {}
for file in os.listdir(".\\contract_data"):
    if file.endswith(".csv"):
        contract_df = pd.read_csv(".\\contract_data\\"+file)
        contract_df.drop_duplicates(subset=['timestamp'], keep='last')
        contract_df["timestamp"] = pd.to_datetime(contract_df["timestamp"])
        
        file = file.replace("-", "/")
        file = file.replace(".csv", "")
        contract_data[file] = contract_df
        
def getcontractvalue(contract_name, trade_timestamp):
    temp_df = contract_data[contract_name]
    t = temp_df.loc[temp_df['timestamp'] == trade_timestamp]["settle_price"]
    return t[t.index[0]]

In [246]:
trade_timestamp = pd.to_datetime("01/03/2017")
trade_df = df_exec_records.loc[df_exec_records['trade_id'] == 0]

dte = (trade_df["expiration_date"][0] - trade_timestamp).days

contract00_val = getcontractvalue(df_exec_records["option_contract"][0],trade_timestamp)
contract01_val = getcontractvalue(df_exec_records["option_contract"][1],trade_timestamp)
contract02_val = getcontractvalue(df_exec_records["option_contract"][2],trade_timestamp)

contract00 = option(df_exec_records["strike"][0],contract00_val,'P', df_exec_records["units"][0],df_exec_records["action"][0], trade_df["expiration_date"][0])
contract01 = option(df_exec_records["strike"][1],contract01_val,'P', df_exec_records["units"][1],df_exec_records["action"][1], trade_df["expiration_date"][1])
contract02 = option(df_exec_records["strike"][2],contract02_val,'P', df_exec_records["units"][2],df_exec_records["action"][2], trade_df["expiration_date"][2])

options = [contract00, contract01, contract02]

futures_prices = [54.12,55.05,55.49,55.68,53.72,52.53,53.87,54.65,53.95,54.03,52.66]

vals = calc_vals(options)

def renderTplusLines(trade_timestamp, contracts):
    theoVals = calc_theoVals(contracts, trade_timestamp)

    risk_plot = figure(title="Risk Graph", plot_height=500, plot_width=700)
    r = risk_plot.line(underlying, vals, line_width=1.5)
    r = risk_plot.line(underlying, theoVals, line_color="orange")
    risk_plot.add_tools(HoverTool(tooltips=[
            ('underlying','@x'),
            ('pnl','@y')
        ],
        formatters={
            'underlying':'printf',

        }
    ))

    show(risk_plot, notebook_handle=True)

    
contract00.update(contract00_val, dte, futures_prices[0])
contract01.update(contract01_val, dte, futures_prices[0])
contract02.update(contract02_val, dte, futures_prices[0])

renderTplusLines(trade_timestamp, options)